In [26]:
# For testing
import os
import re
from ast_helper import wrapper_get, get_call_tree
test_path = 'D:\Code\AC2\AC2\src\main\\test\class.py'
# # test_path = os.path.abspath(test_path)
code = open(test_path, 'r').read()

import ast
import json

def classname(cls):
    return cls.__class__.__name__

def jsonify_ast(node, level=0):
    fields = {}
    try:
        for k in node._fields:
            fields[k] = '...'
            v = getattr(node, k)
            if isinstance(v, ast.AST):
                if v._fields:
                    fields[k] = jsonify_ast(v)
                else:
                    fields[k] = classname(v)

            elif isinstance(v, list):
                fields[k] = []
                for e in v:
                    fields[k].append(jsonify_ast(e))

            elif isinstance(v, str):
                fields[k] = v

            elif isinstance(v, int) or isinstance(v, float):
                fields[k] = v

            elif v is None:
                fields[k] = None

            else:
                fields[k] = 'unrecognized'

        ret = { classname(node): fields }
        return ret
    except:
        ret = { classname(node): fields }

def make_ast(code):
    new_code = re.sub(r'print.*(\\\n.*)+', 'print("pass")\n',  code)
    
    new_code = re.sub(r'print.*\n', 'print("pass")\n',  new_code)
 
    new_code = re.sub(r'except.*:', 'except:',  new_code)
 
    # new_code = re.sub(r'(.*)', '()',  new_code)

    tree = ast.parse(code)
    return jsonify_ast(tree)
    



In [27]:
new_tree = make_ast(code)

In [28]:
new_tree

{'Module': {'body': [{'ClassDef': {'name': 'Person',
     'bases': [],
     'keywords': [],
     'body': [{'FunctionDef': {'name': '__init__',
        'args': {'arguments': {'posonlyargs': [],
          'args': [{'arg': {'arg': 'self',
             'annotation': None,
             'type_comment': None}},
           {'arg': {'arg': 'name', 'annotation': None, 'type_comment': None}},
           {'arg': {'arg': 'age', 'annotation': None, 'type_comment': None}}],
          'vararg': None,
          'kwonlyargs': [],
          'kw_defaults': [],
          'kwarg': None,
          'defaults': []}},
        'body': [{'Assign': {'targets': [{'Attribute': {'value': {'Name': {'id': 'self',
                'ctx': 'Load'}},
              'attr': 'name',
              'ctx': 'Store'}}],
           'value': {'Name': {'id': 'name', 'ctx': 'Load'}},
           'type_comment': None}},
         {'Assign': {'targets': [{'Attribute': {'value': {'Name': {'id': 'self',
                'ctx': 'Load'}},
     

In [3]:

#     return new_data

In [29]:
wrapper_get(new_tree, 'test')

__init__


{'test': [('__init__', 'funcDef'), ('Person', 'func'), ('print', 'func')]}

In [120]:
class_names

['Person']

In [121]:
new_data

{'test': [('Person', 'classDef'), ('Person', 'func')]}

In [116]:
def retrieve_inner_assign_col(obj, from_):
    keys = list(obj.keys())    
    keys2 = list(obj[keys[0]]['value'].keys())
    new_obj = obj[keys[0]]['value']
    key_ = keys2[0]
    
    if key_ == 'Call':
        new_obj = new_obj['Call']
        if list(new_obj.keys())[0] == "func":
            new_obj = new_obj['func']
            
            
#             print(new_obj.keys())
            
            
            if list(new_obj.keys())[0] == "Attribute":
                if 'Name' in list(new_obj['Attribute']['value'].keys()):
                    from_2 = new_obj['Attribute']['value']['Name']['id']
                    func_name_ = obj[keys[0]]['value']['Call']['func']['Attribute']['attr']
                    # print(func_name_, "from", from_2,"called in", from_)

#                         if from_ in new_data.keys():
#                             if (from_2, 'module') not in new_data[from_]:
#                                 new_data[from_].append((from_2, 'module'))
#                         else:
#                             new_data[from_] = [(from_2, 'module')]

                    if from_2 in new_data.keys():
                        if (func_name_, 'func') not in new_data[from_2]:
                            new_data[from_2].append((func_name_, 'func'))
                    else:
                        new_data[from_2] = [(func_name_, 'func')]
                elif 'Attribute' in list(new_obj['Attribute']['value'].keys()):
                    pass
                    # print("\n\n",new_obj['Attribute']['value']['Attribute'].keys(), "\n\n")

            elif list(new_obj.keys())[0] == "Name":
                pass
                # print(new_obj['Name']['id'], "called from", from_)
                
                if from_ in new_data.keys():
                    if (new_obj['Name']['id'], 'func') not in new_data[from_]:
                        new_data[from_].append((new_obj['Name']['id'], 'func'))
                else:
                    new_data[from_] = [(new_obj['Name']['id'], 'func')]


In [137]:
def get_collab_tree(objects, par):
    for i in objects:
        keys = list(i.keys())
        print(keys)
        type_ = keys[0]
        print(type_)
        
        if type_ =='ClassDef':
            print(i['ClassDef']['name'])
            class_names.append(i['ClassDef']['name'])
            new_data[par] = [(i['ClassDef']['name'], 'classDef')]
#         if keys[0] == 'FunctionDef':
#             # print(i['FunctionDef']['name'], "defined in", par)
#             retrieve_inner_functions(i, par)
#         elif keys[0] == 'Expr':
# #             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
#             retrieve_inner_expr(i, par)
        elif keys[0] == 'Assign':
            pass
#             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
            retrieve_inner_assign_col(i, par)
#         elif keys[0] == 'With':
# #             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
#             retrieve_inner_with(i, par)
#         elif keys[0] == 'ImportFrom':
# #             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
#             retrieve_inner_imp_fro(i, par)
#         elif keys[0] == 'Import':
# #             print(i['Expr']['value']['Call']['func']['Name']['ctx'])
#             retrieve_inner_imp(i, par)
#         else:
#             if 'body' in list(i[keys[0]].keys()):
#                 get_call_tree(i[keys[0]]['body'], par)